<a href="https://colab.research.google.com/github/kepsgurih/DBJson/blob/main/RDP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title **Membuat User**
#@markdown Masukan **username** dan **password**

username = "user" #@param {type:"string"}
password = "root" #@param {type:"string"}

print("Membuat user dan konfigurasi sistem")

# Creation of user
! sudo useradd -m $username &> /dev/null

# Add user to sudo group
! sudo adduser $username sudo &> /dev/null
    
# Set password of user to 'root'
! echo '$username:$password' | sudo chpasswd

# Change default shell from sh to bash
! sed -i 's/\/bin\/sh/\/bin\/bash/g' /etc/passwd

print("User telah dibuat")

In [ ]:
#@title **Google Remote Desktop**
#@markdown  Membutuhkan waktu 3-5 Menit

#@markdown  Buka http://remotedesktop.google.com/headless dan salin kode **CRP** dan tempel dikolom ini:

CRP = "DISPLAY= /opt/google/chrome-remote-desktop/start-host --code=\"4/0AY0e-g5o8r7qOvO4LADp_Ou5YIagtld5zjZPI2hHe2svMP0qaUz6pdNUVqeIhIMJp9jQ9A\" --redirect-url=\"https://remotedesktop.google.com/_/oauthredirect\" --name=$(hostname)" #@param {type:"string"}

def CRD():
    with open('install.sh', 'w') as script:
        script.write("""#! /bin/bash

b='\033[1m'
r='\E[31m'
g='\E[32m'
c='\E[36m'
endc='\E[0m'
enda='\033[0m'

printf "\n\n$c$b    Memproses sistem $endc$enda" >&2
if sudo apt-get update &> /dev/null
then
    printf "\r$g$b    Sistem telah dimuat $endc$enda\n" >&2
else
    printf "\r$r$b    Sistem Error $endc$enda\n" >&2
    exit
fi

printf "\n$g$b    Memasang Chrome Remote Desktop $endc$enda" >&2
{
    wget https://dl.google.com/linux/direct/chrome-remote-desktop_current_amd64.deb
    sudo dpkg --install chrome-remote-desktop_current_amd64.deb
    sudo apt install --assume-yes --fix-broken
} &> /dev/null &&
printf "\r$c$b    Chrome Remote Desktop Telah dipasang $endc$enda\n" >&2 ||
{ printf "\r$r$b   Chrome Remote Desktop Gagal dipasang $endc$enda\n" >&2; exit; }
sleep 3

printf "$g$b    Memasang Desktop Environment $endc$enda" >&2
{
    sudo DEBIAN_FRONTEND=noninteractive \
        apt install --assume-yes xfce4 desktop-base
    sudo bash -c 'echo "exec /etc/X11/Xsession /usr/bin/xfce4-session" > /etc/chrome-remote-desktop-session'  
    sudo apt install --assume-yes xscreensaver
    sudo systemctl disable lightdm.service
} &> /dev/null &&
printf "\r$c$b    Desktop Environment Telah dipasang $endc$enda\n" >&2 ||
{ printf "\r$r$b   Desktop Environment Gagal dipasang $endc$enda\n" >&2; exit; }
sleep 3

printf "$g$b    Memasang Google Chrome $endc$enda" >&2
{
    wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
    sudo dpkg --install google-chrome-stable_current_amd64.deb
    sudo apt install --assume-yes --fix-broken
} &> /dev/null &&
printf "\r$c$b    Google Chrome Telah di install $endc$enda\n" >&2 ||
printf "\r$r$b    Google Chrome Gagal dipasang $endc$enda\n" >&2
sleep 3

printf "$g$b    Memasang kernel $endc$enda" >&2
if sudo apt install nautilus nano -y &> /dev/null
then
    printf "\r$c$b    Kernel Telah dipasang $endc$enda\n" >&2
else
    printf "\r$r$b    Kernel Gagal dipasang $endc$enda\n" >&2
fi
sleep 3

printf "\n$g$b    Pemasangan Selesai $endc$enda\n\n" >&2""")

    ! chmod +x install.sh
    ! ./install.sh

    # Adding user to CRP group
    ! sudo adduser $username chrome-remote-desktop &> /dev/null

    # Finishing Work
    ! su - $username -c """$CRP"""

    print("Finished Succesfully")

try:
    if username:
        if CRP == "" :
            print("Masukan kode CRP Google handler")
        else:
            CRD()
except NameError:
    print("Username tidak ditemukan")
    print("Buat user terlebih dahulu")

In [ ]:
#@title **Google Drive Mount**
#@markdown Google Drive used as Persistance HDD for files.<br>
#@markdown Mounted at `user` Home directory inside drive folder
#@markdown (If `username` variable not defined then use root as default).

def MountGDrive():
    from google.colab import drive

    ! runuser -l $user -c "yes | python3 -m pip install --user google-colab"  > /dev/null 2>&1

    mount = """from os import environ as env
from google.colab import drive

env['CLOUDSDK_CONFIG']  = '/content/.config'
drive.mount('{}')""".format(mountpoint)

    with open('/content/mount.py', 'w') as script:
        script.write(mount)

    ! runuser -l $user -c "python3 /content/mount.py"

try:
    if username:
        mountpoint = "/home/"+username+"/drive"
        user = username
except NameError:
    print("username variable not found, mounting at `/content/drive' using `root'")
    mountpoint = '/content/drive'
    user = 'root'

MountGDrive()